In [1]:
import glob
import re
import spacy
import unicodedata
import csv

import pandas as pd

from collections import defaultdict
from minicons.utils import find_pattern
from spacy.lang.en import English
from tqdm import tqdm

import utils
import config
import minicons.utils as mu

from nltk.tokenize import sent_tokenize

from minicons import utils as mu
import inflect
from constructions import AANN

inflector = inflect.engine()
import editors

In [2]:
nlp = English()
tokenizer = nlp.tokenizer

def tokenize(string):
    return [t.text for t in tokenizer(string)]

/home/km55359/.conda/envs/kmisra/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
sent_dir = "/home/km55359/rawdata/babylm_data/babylm_100M/sents/"
# sent_dir = "/Users/kanishka/rawdata/babylm-sents-and-postags/"
sents = utils.read_file(f"{sent_dir}/babylm_sents.txt")
postags = utils.read_file(f"{sent_dir}/postags.txt")

sent_tokens = [tokenize(sent) for sent in sents]

In [4]:
non_numerals = ['few', 'dozen', 'couple', 'several', 'many', 'more']

CURRENT_FEW = r'\bDT\s(((HYPH|,)\s))?((((RB|CC|IN)\s)+)?((JJ|JJR|JJS|VBN|RECORD|((NN CC NN |NN HYPH )+(JJ|JJR|JJS|VBN|RECORD)))((\s(HYPH|,))?)\s))+(((RB)\s)+)?(((HYPH|,)\s))?((UH)\s)?(((NN|CC)\s)+)?((CD|FEW)(\s(TO|CC|(HYPH|,))(\s(HYPH|,))?)?\s)+(((HYPH|,)\s))?((JJR|JJ|VBN)\s)?(DT\s)?((NNS|NNPS|(NN\sNNS)|((NN|NNS) IN NNS)))+'

def detect_dtann_basic(sent_toks, pos):
    p_replaced = []
    for st, pt in zip(sent_toks, pos.split(" ")):
        if st.lower() not in non_numerals + ['record']:
            p_replaced.append(pt)
        else:
            if st.lower() in non_numerals:
                p_replaced.append("FEW")
            elif st.lower() in ["record"]:
                p_replaced.append("RECORD")
            else:
                p_replaced.append(pt)
        # if st.lower() in non_numerals:
        #     p_replaced.append("FEW")
        # else:
        #     p_replaced.append(pt)
        # if st.lower() in ["a", "another", "an", "-a", "-an"]:
        #     p_replaced.append("ARTICLE")
        # else:
        #     p_replaced.append(pt)
    postag_seq = " ".join(p_replaced)
    # print(postag_seq)
    searched = re.search(CURRENT_FEW, postag_seq)
    result = (False, "", "")
    if searched:
        span = searched.span()
        construction_pattern = postag_seq[span[0] : span[1]]
        construction_pattern_span = find_pattern(
            construction_pattern.split(), postag_seq.split()
        )
        if sent_toks != []:
            try:
                sent_span = sent_toks[construction_pattern_span[0] : construction_pattern_span[1]]
                result = (True, " ".join(sent_span), construction_pattern)
            except:
                pass
    return result

In [6]:
dtann_idx = []
dtann_entries = []

for i, (s, st, p) in enumerate(tqdm(zip(sents, sent_tokens, postags))):
    # if i in old_idx:
    #     continue
    detected, span, pattern = detect_dtann_basic(st, p)
    if detected:
        dtann_idx.append(i)
        dtann_entries.append((i, span))

0it [00:00, ?it/s]

11632617it [00:42, 274380.69it/s]


In [7]:
len(dtann_entries)

15781

In [8]:
dtann_entries

[(247, 'The last two reasons'),
 (286, 'the final two partial products'),
 (341, 'The first two digits'),
 (342, 'the other two digits'),
 (884, 'a smaller three to four people'),
 (1074, 'these next few videos'),
 (1427, 'the last eight years'),
 (1430, 'the last three years'),
 (1441, 'the last eight years'),
 (2481, 'a good few seasons'),
 (2887, 'the first few moments'),
 (3810, 'the last 2 days'),
 (4892, 'the first three chords'),
 (5600, 'the past about eighteen months'),
 (5726, 'the next few weeks'),
 (6998, 'the last eighteen months'),
 (8172, 'the Iast few weeks'),
 (8473, 'the last three months'),
 (8713, 'the last two years'),
 (9361, 'the last few decades'),
 (10480, 'the top 64 snooker players'),
 (10524, 'the closest 68,000 galaxies'),
 (11143, 'the top 64 snooker players'),
 (11270, 'that non - zero number zero times'),
 (11341, 'the top 32 snooker players'),
 (11371, 'the first nineteen years'),
 (11579, 'the other 9 districts'),
 (12038, 'the future M S two posts'),


In [9]:
with open("../data/babylm-analysis/dtanns.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["sentence_idx", "sentence"])
    for idx, sent in dtann_entries:
        writer.writerow([idx, sent])